In [1]:
!pip install nltk
!pip install tensorflow
!pip install keras
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import tensorflow as tf

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
#imports 
import pandas as pd
import numpy as np
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\dantu\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dantu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dantu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dantu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
#loading the dataset
dataset = pd.read_csv('ML_Assignment3_dataset.csv', header=0)

In [4]:
tweets=dataset.iloc[0:,2]

In [5]:
labels=dataset.iloc[0:,5]

In [6]:
Counter(labels)

Counter({1: 520, 0: 5004})

In [7]:
dataset.iloc[0:,2]

0                   I've been a very lazy Tweeter lately.
1                         Good morning, beautiful people!
2                                   Happy Canada Day, eh.
3       I just got added to a Twitter list called "Twi...
4       There's a webinar for everything these days. I...
                              ...                        
5519                               Ty is one happy guy rn
5520                                   Great weekend 🔴🔵⚫️
5521                        Them NNS had some cute chirps
5522    Good luck to the boys in cross lake tonight, G...
5523                    しかもしかもいつもより早い電車に乗らなきゃいけないとかなんなの……
Name: Tweets, Length: 5524, dtype: object

# Pre-processing 

In [8]:
import re
!pip install emot
import pickle
! pip3 install googletrans==3.1.0a0

from emot.emo_unicode import UNICODE_EMOJI # For emojis
from emot.emo_unicode import EMOTICONS_EMO
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)




Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [9]:
#Defining a function for preprocessed tweets

import sys  
!{sys.executable} -m pip install contractions
import contractions
!pip install ftfy
import ftfy
import nltk
nltk.download('omw-1.4')
#from google_trans_new import google_translator
from googletrans import Translator, constants
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

translator = Translator()
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_tweet(tweet):
    
    translation = translator.translate(tweet)
    tweet_nomentions=re.sub("@[A-Za-z0-9_]+","", translation.text)
    tweet_nohashtags=re.sub("#[A-Za-z0-9_]+","", tweet_nomentions)
    tweet_noemoticons=re.sub(r'(\:\w+\:|\<[\/\\]?3|[\(\)\\\D|\*\$][\-\^]?[\:\;\=]|[\:\;\=B8][\-\^]?[3DOPp\@\$\*\\\)\(\/\|])(?=\s|[\!\.\?]|$)',"", tweet_nohashtags)
    tweet_noemojis=emoji_pattern.sub(r'', tweet_noemoticons)
    tweet_nopunctuations=re.sub(r'[^\w\s]', ' ', tweet_noemojis)
    tweet_expanded=contractions.fix(tweet_nopunctuations)
    tweet_noencoding = ftfy.fix_text(tweet_expanded)
    word_tokens = word_tokenize(tweet_noencoding)
    lemmatized_tweet = ' '.join([lemmatizer.lemmatize(w) for w in word_tokens])
    word_tokens = word_tokenize(lemmatized_tweet)
    nonstopwords = [word for word in word_tokens if not word.lower() in stop_words]
    preprocessed_tweet = ' '.join(nonstopwords)
    
    return preprocessed_tweet


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dantu\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [10]:
tweets=tweets.tolist()
preprocessed_tweets=list()

In [11]:
type(preprocessed_tweets)

list

In [12]:
for t in tweets:
    preprocessed_tweets.append(preprocess_tweet(t))
    

In [13]:
preprocessed_tweets

['lazy Tweeter lately',
 'Good morning beautiful people',
 'Happy Canada Day eh',
 'got added Twitter list called Twitter Legends Bahahahaha',
 'webinar everything day market overloaded learn stuff really easy path knowledge',
 'Buy stuff Look new thing Sign junk',
 'realized 15 year since graduated COLLEGE',
 'thing mid forty see anything close anymore knee shot cute haircut',
 'Twitter day',
 'Hey Twitter shaking',
 'Good morning Excited teach Project Management morning',
 'Good morning beautiful people FRIDAY',
 'Apparently hell froze wa sleeping',
 'Good morning beautiful people',
 'Happy May Fourth Eve',
 'allergy shot little year tell life altering Pollen everywhere feel dandy',
 'Welp yard work going',
 'Good morning beautiful people gorgeous day Nation Capital',
 'Hey looking borrow mannequin',
 'football thing hockey stuff boxing thing news right Holy crikey could care le sport',
 'Tomorrow Friday meeting accountant woohoo Friday Ish',
 'send email need create experience',
 'L

In [14]:
#declaring constants
MAX_WORDS = 20000
DIMENSION=300
TRAIN=0.6
TEST=0.2
VAL=0.2

In [15]:
from keras.utils import pad_sequences
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
#splitting the tweets into unique words
tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(preprocessed_tweets)
#converting tweets into sequence of tokens
sequences = tokenizer.texts_to_sequences(preprocessed_tweets)
#assigning indexes to each token
word_index = tokenizer.word_index
padded_data = pad_sequences(sequences,maxlen=140)

In [16]:
padded_data

array([[   0,    0,    0, ...,  929, 1696,  769],
       [   0,    0,    0, ...,   38,  112,    5],
       [   0,    0,    0, ...,  145,    7, 1220],
       ...,
       [   0,    0,    0, ..., 9676,  301, 9677],
       [   0,    0,    0, ...,   30,    3,  190],
       [   0,    0,    0, ..., 1167,  696, 2123]])

In [17]:
len(word_index)

9677

In [18]:
padded_data.shape

(5524, 140)

In [61]:
print(padded_data)

[[   0    0    0 ...  929 1696  769]
 [   0    0    0 ...   38  112    5]
 [   0    0    0 ...  145    7 1220]
 ...
 [   0    0    0 ... 9676  301 9677]
 [   0    0    0 ...   30    3  190]
 [   0    0    0 ... 1167  696 2123]]


In [19]:
#Load Pretrained Word2Vec Model 
import gensim
from gensim.models import KeyedVectors
pretrained_file= 'GoogleNews-vectors-negative300.bin.gz'
word2vec_model = KeyedVectors.load_word2vec_format(pretrained_file, binary=True)

In [20]:
#Embedding matrix
words = min(20000, len(word_index))+1
skip=0
embedding_matrix = np.zeros((words,DIMENSION))
for word,index in word_index.items():
  try:
    embedding_vector=word2vec_model[word]
  except:
    skip+=1
    pass

  if embedding_vector is not None:
    embedding_matrix[index]=embedding_vector


In [59]:
embedding_matrix.shape

(9678, 300)

Train Test Split

In [22]:
X=dataset.iloc[:,2].values
Y=dataset.iloc[:,5].values

In [23]:
Y

array([1, 0, 0, ..., 0, 0, 0], dtype=int64)

In [24]:
lables_array = [x for x in dataset.iloc[0:,5]]
lables=np.array(lables_array)

In [25]:
np.random.seed(42)
permuted_data = np.random.permutation(len(padded_data))
index_train = permuted_data[:int(len(padded_data)*(TRAIN))]
train_data= padded_data[index_train]
train_labels =labels[index_train]
index_test = permuted_data[int(len(padded_data)*(TRAIN)):int(len(padded_data)*(TRAIN+TEST))]
test_data =padded_data[index_test]
test_labels=labels[index_test]
index_val =permuted_data[int(len(padded_data)*(TRAIN+TEST)):]
val_data =padded_data[index_val]
val_labels=labels[index_val]

In [62]:
Counter(train_labels)

Counter({0: 3006, 1: 308})

RNN Model Implementation

In [76]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
model1=Sequential()
model1.add(Embedding(input_dim=words,output_dim=DIMENSION,weights=[embedding_matrix])) #embedding layer
model1.add(SimpleRNN(128,activation='relu',return_sequences=True))
model1.add(SimpleRNN(256,activation='relu'))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss="binary_crossentropy",metrics=['accuracy'],optimizer='adam')
print(model1.summary())

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, None, 300)         2903400   
                                                                 
 simple_rnn_18 (SimpleRNN)   (None, None, 128)         54912     
                                                                 
 simple_rnn_19 (SimpleRNN)   (None, 256)               98560     
                                                                 
 dense_12 (Dense)            (None, 1)                 257       
                                                                 
Total params: 3,057,129
Trainable params: 3,057,129
Non-trainable params: 0
_________________________________________________________________
None


In [77]:
#training the model
early_stop = EarlyStopping(monitor='val_loss', patience=3)
hist = model1.fit(train_data, train_labels, \
        validation_data=(val_data, val_labels), \
        epochs=10, batch_size=40, shuffle=True, \
        callbacks=[early_stop])

Epoch 1/10
83/83 [==============================] - 27s 221ms/step - loss: 0.3037 - accuracy: 0.9025 - val_loss: 0.2775 - val_accuracy: 0.9014
Epoch 2/10
83/83 [==============================] - 18s 216ms/step - loss: 0.1860 - accuracy: 0.9140 - val_loss: 0.2964 - val_accuracy: 0.9032
Epoch 3/10
83/83 [==============================] - 17s 210ms/step - loss: 26286274560.0000 - accuracy: 0.9267 - val_loss: 0.3146 - val_accuracy: 0.9068
Epoch 4/10
83/83 [==============================] - 18s 212ms/step - loss: 0.0783 - accuracy: 0.9707 - val_loss: 0.4010 - val_accuracy: 0.8860


In [78]:
#predicting the labels for test dataset in RNN Model
from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score
pred_labels = model.predict(test_data)
pred_labels = np.round(pred_labels.flatten())
print(classification_report(test_labels, pred_labels))

35/35 [==============================] - 2s 50ms/step
              precision    recall  f1-score   support

           0       0.93      0.92      0.93      1002
           1       0.31      0.36      0.33       103

    accuracy                           0.87      1105
   macro avg       0.62      0.64      0.63      1105
weighted avg       0.88      0.87      0.87      1105



In [79]:
pred_labels


array([1., 0., 0., ..., 0., 0., 0.], dtype=float32)

CNN+LSTM Model Implementation

In [80]:
model2 = Sequential()
model2.add(Embedding(len(embedding_matrix), DIMENSION, weights=[embedding_matrix], 
                            input_length=140, trainable=False)) #embedding layer 
model2.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')) #Convolutional layer
model2.add(MaxPooling1D(pool_size=2)) 
model2.add(Dropout(0.2))
model2.add(LSTM(300)) #LSTM layer
model2.add(Dropout(0.2))
model2.add(Dense(1, activation='sigmoid'))

In [81]:
model2.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])

In [82]:
#training the model
from keras.callbacks import *
early_stop = EarlyStopping(monitor='val_loss', patience=3)
hist = model2.fit(train_data, train_labels, \
        validation_data=(val_data, val_labels), \
        epochs=10, batch_size=40, shuffle=True, \
        callbacks=[early_stop])

Epoch 1/10
83/83 [==============================] - 25s 223ms/step - loss: 0.3345 - acc: 0.9071 - val_loss: 0.2966 - val_acc: 0.9014
Epoch 2/10
83/83 [==============================] - 18s 214ms/step - loss: 0.2604 - acc: 0.9071 - val_loss: 0.2746 - val_acc: 0.9014
Epoch 3/10
83/83 [==============================] - 17s 208ms/step - loss: 0.2265 - acc: 0.9086 - val_loss: 0.2802 - val_acc: 0.8995
Epoch 4/10
83/83 [==============================] - 17s 209ms/step - loss: 0.1960 - acc: 0.9158 - val_loss: 0.2960 - val_acc: 0.9041
Epoch 5/10
83/83 [==============================] - 18s 212ms/step - loss: 0.1606 - acc: 0.9303 - val_loss: 0.3284 - val_acc: 0.9050


In [83]:
#predicting the labels for test dataset for CNN+LSTM Model
pred_labels = model2.predict(test_data)
pred_labels = np.round(pred_labels.flatten())
print(classification_report(test_labels, pred_labels))

35/35 [==============================] - 3s 68ms/step
              precision    recall  f1-score   support

           0       0.92      0.99      0.95      1002
           1       0.59      0.16      0.25       103

    accuracy                           0.91      1105
   macro avg       0.76      0.57      0.60      1105
weighted avg       0.89      0.91      0.89      1105



In [84]:
pred_labels

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)